# Testing API

First, open a terminal and run the server with `uv run python -m app.main`.
This will start the server at the localhost on port 8000.


In [47]:
import pandas as pd

sample = pd.read_parquet("../tests/sample_data/sample_data.parquet")

sample['n_labels'] = sample['ramo_direito'].apply(lambda x: len(x))

In [48]:
import requests

response = requests.post(
    "http://localhost:8000/api/pecas/1",
    json={"texto": ""}
)

print(f"Status Code: {response.status_code}")
print(f"Response: {response.json()}")



Status Code: 200
Response: {'id': 1, 'ramo_direito': ['DIREITO ADMINISTRATIVO E OUTRAS MATÉRIAS DE DIREITO PÚBLICO']}


In [49]:
# Create an empty list to store results
results = []

# Iterate through each row and make POST request
for idx, row in sample.iterrows():
    response = requests.post(
        f"http://localhost:8000/api/pecas/{idx}",
        json={"texto": row['texto_bruto']}
    )
    
    # Store the result along with the original text
    results.append(response.json())

In [70]:
res = pd.DataFrame(results)
res.rename(columns={'ramo_direito': 'predicted'}, inplace=True)
res['target'] = sample['ramo_direito'].values
res['n_labels_pred'] = res.apply(lambda x: len(x['predicted']), axis=1)
res['n_labels_target'] = res.apply(lambda x: len(x['target']), axis=1)

res['perfect_match'] = res.apply(lambda x: all(x['predicted'] == x['target']), axis=1)
subset_accuracy = sum(res['perfect_match'])/sample.shape[0]
print(f"Subset Accuracy (all labels match): {subset_accuracy:.2%}")

res['partial_match'] = res.apply(lambda x: any(label in x['target'] for label in x['predicted']), axis=1)
partial_accuracy = sum(res['partial_match'])/sample.shape[0]
print(f"Partial Accuracy (at least one match): {partial_accuracy:.2%}")

# Compute number of correct matches for each label
res['correct_matches'] = res.apply(lambda x: sum(label in x['target'] for label in x['predicted']), axis=1)
res['missing_labels'] = res.apply(lambda x: sum(label not in x['predicted'] for label in x['target']), axis=1)

Subset Accuracy (all labels match): 54.55%
Partial Accuracy (at least one match): 100.00%


In [68]:
res

,id,predicted,target,n_labels_pred,n_labels_target,perfect_match,partial_match,correct_matches,missing_labels
0,2193,[DIREITO ADMINISTRATIVO E OUTRAS MATÉRIAS DE D...,[DIREITO ADMINISTRATIVO E OUTRAS MATÉRIAS DE D...,1,1,True,True,1,0
1,7099,[DIREITO PROCESSUAL CIVIL E DO TRABALHO],[DIREITO PROCESSUAL CIVIL E DO TRABALHO],1,1,True,True,1,0
2,9145,[DIREITO TRIBUTÁRIO],[DIREITO TRIBUTÁRIO],1,1,True,True,1,0
3,8582,[DIREITO ADMINISTRATIVO E OUTRAS MATÉRIAS DE D...,[DIREITO ADMINISTRATIVO E OUTRAS MATÉRIAS DE D...,2,2,True,True,2,0
4,2592,"[DIREITO PENAL, DIREITO PROCESSUAL PENAL]","[DIREITO PENAL, DIREITO PROCESSUAL PENAL]",2,2,True,True,2,0
5,8199,"[DIREITO PENAL, DIREITO PROCESSUAL PENAL]","[DIREITO PENAL, DIREITO PROCESSUAL PENAL]",2,2,True,True,2,0
6,5030,[DIREITO ADMINISTRATIVO E OUTRAS MATÉRIAS DE D...,"[DIREITO PREVIDENCIÁRIO, DIREITO ADMINISTRATIV...",3,3,False,True,3,0
7,7851,[DIREITO ADMINISTRATIVO E OUTRAS MATÉRIAS DE D...,[DIREITO ADMINISTRATIVO E OUTRAS MATÉRIAS DE D...,2,2,True,True,2,0
8,9542,[DIREITO ADMINISTRATIVO E OUTRAS MATÉRIAS DE D...,"[DIREITO DO TRABALHO, DIREITO TRIBUTÁRIO, DIRE...",3,3,False,True,3,0
9,1409,[DIREITO DA SAÚDE],[DIREITO DA SAÚDE],1,1,True,True,1,0
